In [21]:
# Импортируем библиотеки
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from sklearn import linear_model #линейные моделиё
from sklearn import ensemble #ансамбли

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

In [3]:
# Загрузим датасет из библиотеки
data = load_diabetes(as_frame=True)
# Создаем матрицу наблюдений
X = data['frame'].drop('target', axis=1)
# Создаем вектор правильных ответов
y = data['target']
 
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [4]:
# Разделим выборку на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

# Посмотрим на размерности выборок
print(f'Размерность обучающей выборки {X_train.shape}')
print(f'Размерность тестовой выборки {X_test.shape}')

Размерность обучающей выборки (296, 10)
Размерность тестовой выборки (146, 10)


**Задание 3.5**<br>
Для выполнения задания используйте набор данных о диабете, который представлен в ноутбуке с примерами.

Постройте стекинг из следующих базовых моделей:

Ридж-регрессия (RidgeCV());
Линейная регрессия.
В качестве метамодели используйте случайный лес с количеством деревьев 100, максимальной глубиной 10, все параметры для базовых моделей стандартные. Для всех алгоритмов параметр random_state=42.

Сделайте предсказание целевой метки для тестового набора данных. Рассчитайте метрику MSE для набора данных и запишите её в качестве ответа с точностью до одного знака после запятой.

MSE на тестовой выборке:

In [21]:
# Создаем список кортежей вида: (наименование модели, модель)

from sklearn.linear_model import LinearRegression
estimators = [
    ('rr', RidgeCV()),
    ('lr',  LinearRegression())
]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
)
#Обучаем модель
reg.fit(X_train, y_train)

StackingRegressor(estimators=[('rr', RidgeCV()), ('lr', LinearRegression())],
                  final_estimator=RandomForestRegressor(max_depth=10,
                                                        random_state=42))

3549.8

In [22]:
# Формируем предсказание для тестовой выборки
y_pred_stack = reg.predict(X_test)
# Оцениваем качество по метрике MSE
print(f'Качество предсказания по MSE для стекинга {round(mean_squared_error(y_test, y_pred_stack), 1)}')

Качество предсказания по MSE для стекинга 3534.7


**Задание 4.1**
Измените количество базовых моделей до 50 и 100. В качестве ответа приведите полученные результаты, округленные до второго знака после запятой. Остальные параметры оставьте неизменными.

Качество модели по MSE для 50 базовых моделей:

In [6]:
from sklearn.ensemble import AdaBoostRegressor

# Создаем объект класса дерева решений
dt = DecisionTreeRegressor(
    max_depth=3, #максимальная глубина дерева
    random_state=42 #датчик генератора случайных чисел
)
# Создаем объект класса AdaBoost
ada = AdaBoostRegressor(
    estimator=dt, #базовая модель
    n_estimators=100, #количество моделей в ансамбле
    random_state=42 #датчик генератора случайных чисел
)
# Обучаем модель
ada.fit(X_train, y_train)

# Формируем предсказание для тестовой выборки
ada_pred  = ada.predict(X_test)

# Оцениваем качество по метрике MSE
print(f'Качество предсказания по MSE для AdaBoost {round(mean_squared_error(y_test, ada_pred),2)}')


Качество предсказания по MSE для AdaBoost 3101.68


**Задание 4.5**
Измените количество базовых моделей с 10 до 50 и 100. В качестве ответа приведите полученные результаты, округленные до второго знака после запятой. Если второе число после запятой равно нулю, округлите до одного знака: например 10.10 необходимо округлить до 10.1. Остальные параметры оставьте неизменными.

In [9]:
from sklearn.ensemble import GradientBoostingRegressor

# Создаем объект класса градиентный бустинг
gb = GradientBoostingRegressor(
    max_depth=3, #максимальная глубина дерева
    n_estimators=100, #количество деревьев в ансамбле
    random_state=42 #датчик генератора случайных чисел
)

# Обучаем модель
gb.fit(X_train, y_train)

# Формируем предсказание для тестовой выборки
gb_pred  = gb.predict(X_test)

# Оцениваем качество по метрике MSE
print(f'Качество предсказания по MSE для GradientBoostingRegressor {round(mean_squared_error(y_test, gb_pred),2)}')

Качество предсказания по MSE для GradientBoostingRegressor 3286.02


**Задание 4.6**
Обучите на тренировочной выборке (X_train, y_train) модель градиентного бустинга, состоящего из 200 деревьев, максимальная глубина каждого из которых равна 3. Минимальное число объектов в листе — 10. Темп обучения возьмите равный 0.01. Параметр random_state установите в значение 42.

Сделайте предсказание качества вина для тренировочного и тестового набора данных. Рассчитайте метрику accuracy для тестового набора (X_test, y_test) и запишите её в качестве ответа с точностью до двух знаков после запятой.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

# Читаем исходные данные к задаче классификации
df = pd.read_csv('data/winequality-red.csv', sep = ';')
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [11]:
# Создаем матрицу наблюдений 
X = df.drop(['quality'],axis = 1)
# Создаем вектор признаков для бинарной классификации
y = (df['quality'] >5).astype(int)

In [12]:
# Разделим выборку на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
# Посмотрим на размерности выборок
print(f' Размерность обучающей выборки {X_train.shape}')
print(f' Размерность тестовой выборки {X_test.shape}')

 Размерность обучающей выборки (1071, 11)
 Размерность тестовой выборки (528, 11)


In [17]:
# Создаем модель градиентного бустинга
gb = GradientBoostingClassifier(
    loss='log_loss', #функция потерь
    learning_rate=0.2, #темп обучения
    min_samples_leaf=10, #Минимальное число объектов в листе
    n_estimators=500, #число деревьев
    max_depth=3, #максимальная глубина дерева
    random_state=42 #генератор случайных чисел
)
# Обучаем модель
gb.fit(X_train, y_train)
# Формируем предсказание для тестовой выборки
y_pred = gb.predict(X_test)
# Посмотрим на основные метрики классификации
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.76      0.77       238
           1       0.81      0.83      0.82       290

    accuracy                           0.80       528
   macro avg       0.79      0.79      0.79       528
weighted avg       0.80      0.80      0.80       528



**Задание 6.1**
На основании датасета предсказания рейтинга красного вина, с которым вы работали в разделе о предобработке данных в предыдущем юните (файл Red.csv ), вам предстоит выполнить следующее:

Добавить в пайплайн, созданный вами в предыдущем юните, обработку столбца 'Region'. Кодирование 'Region' произвести с использованием OrdinalEncoder. Стандартизацию столбца 'Price' и кодирование столбца 'Country' осуществить в соответствии с примером, представленным в юните 5.

Обучить пайплайн на тренировочном наборе данных (файл Red.csv ) и оценить качество модели по метрике RMSE на тестовом наборе (файл Red_test.csv ).
Зафиксировать random_state=42.
Сохранить пайплайн в файл pipeline_wine.pkl.
В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [62]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import make_column_transformer


In [63]:
df_wine = pd.read_csv('data/Red.csv')
df_wine_test = pd.read_csv('data/Red_test.csv')

In [64]:
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8666 entries, 0 to 8665
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8666 non-null   object 
 1   Country          8666 non-null   object 
 2   Region           8666 non-null   object 
 3   Winery           8666 non-null   object 
 4   Rating           8666 non-null   float64
 5   NumberOfRatings  8666 non-null   int64  
 6   Price            8666 non-null   float64
 7   Year             8666 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 541.8+ KB


In [65]:
def rmse(y_hat, y):
    return mean_squared_error(y_hat, y, squared = False)

In [66]:
ct = make_column_transformer(
     (OrdinalEncoder(), ['Region']),
    (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']))
print(ct)

ColumnTransformer(transformers=[('ordinalencoder', OrdinalEncoder(),
                                 ['Region']),
                                ('standardscaler', StandardScaler(), ['Price']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['Country'])])


In [67]:
pipeline = Pipeline([('ct', ct), ('rf', RandomForestRegressor(random_state=42))])

In [68]:
X = df_wine[['Region', 'Price', 'Country']]
y = df_wine['Rating']

X_test = df_wine_test[['Region', 'Price', 'Country']]
y_test = df_wine_test['Rating']

In [69]:
pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor(random_state=42))])

In [70]:
y_pred = pipeline.predict(X_test)
print(f'Качество по метрике R2: { round(r2_score(y_test, y_pred),4)}')
print(f'Качество по RSME: {round((mean_squared_error(y_test, y_pred)**0.5),4)}')

Качество по метрике R2: 0.9393
Качество по RSME: 0.0765


In [98]:
import joblib
joblib.dump(pipeline, 'pipeline.pkl')


['pipeline.pkl']

In [101]:
pipeline_loaded = joblib.load('pipeline.pkl')
print(pipeline_loaded)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf',
                 StackingRegressor(estimators=[('lr', RidgeCV()),
                                               ('dt',
                                                DecisionTreeRegressor(random_state=42))],
                                   final_estimator=RandomForestRegressor(n_estimators=10,
                                                                         random_state=42)))])


**Задание 6.2**
Теперь попробуем изменить параметры случайного леса в пайплайне, полученном в предыдущем задании.

Измените параметр n_estimators в случайном лесу со значения по умолчанию до 200 , используя метод set_params.

В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [71]:
print(pipeline['rf'].n_estimators)

100


In [72]:
pipeline.set_params(rf__n_estimators=200)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf',
                 RandomForestRegressor(n_estimators=200, random_state=42))])

In [74]:
pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf',
                 RandomForestRegressor(n_estimators=200, random_state=42))])

In [93]:
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round((mean_squared_error(y_test, y_pred)**0.5),4)}')

NotFittedError: This StackingRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [102]:
joblib.dump(pipeline, 'pipeline_1.pkl')

['pipeline_1.pkl']

In [103]:
joblib.load('pipeline_1.pkl')

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf',
                 StackingRegressor(estimators=[('lr', RidgeCV()),
                                               ('dt',
                                                DecisionTreeRegressor(random_state=42))],
                                   final_estimator=RandomForestRegressor(n_estimators=10,
                                                                         random_state=42)))])

**Задание 6.3**
Теперь попробуем добавить стекинг в качестве модели в пайплайн.

Вам следует выполнить следующее:

Собрать StackingRegressor:
В качестве базовых моделей возьмите ридж-регрессию RidgeCV() и решающее дерево.
В качестве метамодели возьмите случайный лес с настройками (количество базовых моделей 10).
Все базовые модели стекинга модели должны быть с настройками по умолчанию (кроме random_state).
Зафиксировать random_state=42 (для всех моделей).
Заменить в пайплайне задачи 6.1 случайный лес на StackingRegressor.
Обучить модель на тренировочной выборке.
В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округлённый до второго знака после запятой.

In [77]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

In [79]:
#Создаем список кортежей вида: (наименование модели, модель)
estimators = [
    ('lr', RidgeCV()),
    ('dt',  DecisionTreeRegressor(random_state=42))
]

In [80]:
# Создаем объект класса стекинг
reg = StackingRegressor(
    estimators=estimators, #базовые модели в стекинге
    final_estimator=RandomForestRegressor( #метамодель
        n_estimators=10, #количество деревьев в ансамбле                    
        random_state=42 #датчик генератора случайных чисел
    )
)
# Обучаем модель
# reg.fit(X_train, y_train)

In [91]:
pipeline.set_params(rf = reg)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf',
                 StackingRegressor(estimators=[('lr', RidgeCV()),
                                               ('dt',
                                                DecisionTreeRegressor(random_state=42))],
                                   final_estimator=RandomForestRegressor(n_estimators=10,
                                                                         random_state=42)))])

In [92]:
pipeline.get_params(0)

{'memory': None,
 'steps': [('ct',
   ColumnTransformer(transformers=[('ordinalencoder', OrdinalEncoder(),
                                    ['Region']),
                                   ('standardscaler', StandardScaler(), ['Price']),
                                   ('onehotencoder', OneHotEncoder(),
                                    ['Country'])])),
  ('rf',
   StackingRegressor(estimators=[('lr', RidgeCV()),
                                 ('dt', DecisionTreeRegressor(random_state=42))],
                     final_estimator=RandomForestRegressor(n_estimators=10,
                                                           random_state=42)))],
 'verbose': False}

In [95]:
pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf',
                 StackingRegressor(estimators=[('lr', RidgeCV()),
                                               ('dt',
                                                DecisionTreeRegressor(random_state=42))],
                                   final_estimator=RandomForestRegressor(n_estimators=10,
                                                                         random_state=42)))])

In [97]:
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round((mean_squared_error(y_test, y_pred)**0.5),2)}')

Качество по RSME: 0.18
